In [1]:
from monitor.monitor import Monitor

In [2]:
monitor = Monitor()
print(monitor)

LUNAR LANDER MONITOR PROPERTIES:

	 Q-func Model Path: 		./output/lunar_lander.keras
	 Q-Model Num Actions: 		4

	 Training Data Path: 		./output/latest_buffer_states.npy
	 Output Video Path: 		./output/lunar_lander.mp4
	 Output Video FPS: 			30

	 Monitor Trace Path: 		./output/latest_traces_{}.npy
	 Model Prediction Path: 	./output/latest_preds.npy


In [3]:
monitor.extract_traces_and_preds()

3125/3125 ━━━━━━━━━━━━━━━━━━━━ 1s 160us/step
(100000, 8)
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 1s 163us/step

Agent model Q-values saved to file: ./output/latest_preds.npy with shape: (100000, 4)
Monitor traces for predicted action: 0 saved to file: ./output/latest_traces_0.npy with shape: (21442, 8)
Monitor traces for predicted action: 1 saved to file: ./output/latest_traces_1.npy with shape: (24349, 8)
Monitor traces for predicted action: 2 saved to file: ./output/latest_traces_2.npy with shape: (16319, 8)
Monitor traces for predicted action: 3 saved to file: ./output/latest_traces_3.npy with shape: (37890, 8)
